# Reference inductor

In [ ]:
# donner un pipfile pour la config

from shapeOptInductor import create_plots, update_plots, rot, gen_mesh  # noqa: F401
import ngsolve as ngs
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as plt

## 1 - Geometry and meshing

In [ ]:
e = 1.78e-3
thickness = 1e-2

maxh = 2e-3
mesh = gen_mesh(e, maxh)


XiAir = mesh.MaterialCF({"air": 1})
XiCore = mesh.MaterialCF({"core": 1})
XiCoil = mesh.MaterialCF({"coil": 1})

Draw(1 * XiAir + 2 * XiCoil + 3 * XiCore, mesh, radius=0.02)

## 2 - Computation of magnetic state

In [ ]:
# Frequency
f = 5e4  # Hz
omega = 2 * np.pi * f

# Magnetic
mu0 = 4e-7 * np.pi
mur = 1000
mu_iron = mur * mu0
delta = 0.1
mu_coil = np.exp(-1j * delta) * mu0  #  AC losses in the copper from the imaginary part of the permeability

# Current
nb_turn = 200  # Number of turn in the coil
Is = 2  # Source current intensity
js = nb_turn / 2 * Is / (ngs.Integrate(XiCoil, mesh)) * XiCoil  # Source current density


def magWeakFormComplex(a, a_):
    bf = ngs.grad(a_) * 1 / mu_iron * ngs.grad(a) * ngs.dx("core")
    bf += ngs.grad(a_) * 1 / mu_coil * ngs.grad(a) * ngs.dx("coil")
    bf += ngs.grad(a_) * 1 / mu0 * ngs.grad(a) * ngs.dx("air")
    lf = a_ * js * ngs.dx("coil")
    return bf, lf


def solveStateComplex(fes):
    a, a_ = fes.TnT()
    bf, f = magWeakFormComplex(a, a_)
    K, F = ngs.BilinearForm(fes), ngs.LinearForm(fes)
    K += bf
    F += f
    K.Assemble()
    F.Assemble()
    gf = ngs.GridFunction(fes)
    Kinv = K.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
    gf.vec.data = Kinv * F.vec
    return gf, Kinv


fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
a, _ = solveStateComplex(fes)
Draw(ngs.Norm(a), mesh)

In [ ]:
def Inductance(a, mesh):
    rel = XiAir / mu0 + XiCoil / mu_coil + XiCore / mu_iron
    return 4 * thickness / (2 * Is**2) * ngs.Integrate(rel.real * ngs.Norm(ngs.grad(a)) ** 2, mesh)


def Losses(a, mesh):
    rel = XiCoil / mu_coil
    return 4 * omega / 2 * thickness * ngs.Integrate(rel.imag * ngs.Norm(ngs.grad(a)) ** 2, mesh)


print(f" L = {Inductance(a,mesh) * 1e6 :.2f} µH")
print(f" P_AC = {Losses(a,mesh) :.2f} W")